# Creating the dataset from ATLAS3D

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
import csv

### Loading the two samples

In [2]:
capXV=np.loadtxt('./raw_data/ATLAS3D_CappellariXV_2013.txt',  usecols=np.arange(1, 15))
capXX= np.loadtxt('./raw_data/ATLAS3D_CappellariXX_2013.txt', usecols=np.arange(1, 5))

### Calculating stellar masses and DM fraction

In [12]:
logML = capXX[:,-2]
logL = capXV[:,-1]
R = capXV[:,-6]
M = 10**(logML+logL)
fDM = capXX[:,-3]


### Making the final csv file

In [14]:
# Writing the data into a CSV file
with open('dataset/ATLAS3D.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Writing the name line and the unit line
    writer.writerow(["Name", "M*", "Reff","fDM(Reff)", "sigma_fdm(Reff)"])
    writer.writerow(["None", "[Msun]","[kpc]", "None", "None"])

    # Looping through each element of the data
    for i in range(len(capXV)):
        name = capXV[i,0]
        Mass = M[i]

        fDMeff = fDM[i]
        Reff = R[i]
        sigma_fdmeff = 0 #no errors provided
        # Writing the data into the CSV file
        writer.writerow([name, Mass, Reff, fDMeff, sigma_fdmeff])
        
# Reading the CSV file into a DataFrame
cappellari = pd.read_csv('dataset/ATLAS3D.csv')
cappellari

,Name,M*,Reff,fDM(Reff),sigma_fdm(Reff)
0,NaN,[Msun],[kpc],NaN,NaN
1,1.879,9885530946.56941,1.154,0.12,0.0
2,1.996,12473835142.429434,0.956,0.24,0.0
3,1.806,13835663789.71781,1.396,0.20,0.0
4,2.108,33884415613.920208,1.027,0.11,0.0
...,...,...,...,...,...
254,1.781,7063175542.629622,0.765,0.00,0.0
255,2.125,31117163371.0602,1.018,0.25,0.0
256,1.984,12078138351.067772,0.988,0.50,0.0
257,2.105,27925438412.373363,0.835,0.00,0.0
